In [4]:
# Install dependencies
!pip install -q sentence-transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.5 MB/s eta 

In [5]:
from sentence_transformers import SentenceTransformer
import chromadb
import ipywidgets as widgets
from IPython.display import display

# Load embedding model
model = SentenceTransformer("all-mpnet-base-v2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
client = chromadb.PersistentClient(path="./prompt_db")

try:
    collection = client.get_collection("prompts")
except:
    collection = client.create_collection(name="prompts")


In [7]:
template_prompts = [
    "Summarize a long research paper into concise bullet points.",
    "Generate SEO-friendly blog titles for a given topic.",
    "Create technical interview questions for a data scientist role.",
    "Draft an email apologizing for a shipping delay.",
    "Convert a paragraph into a funny, sarcastic version.",
    "Make a table comparing two algorithms.",
    "Explain supervised learning vs unsupervised learning.",
    "Give me examples of deep learning applications.",
    "Prepare a short story suitable for children aged 8-10.",
    "Turn a technical explanation into a simple analogy."
]

embeddings = [model.encode(p, normalize_embeddings=True) for p in template_prompts]

collection.add(
    documents=template_prompts,
    embeddings=embeddings,
    ids=[f"t{i+1}" for i in range(len(template_prompts))]
)
print("Prompts added successfully!")


Prompts added successfully!


In [14]:
def query_prompt(user_input, n_results=1, threshold=0.7):
    ###################### Get stored prompts and embeddings########################################
    all_docs = collection.get(include=["documents", "embeddings"])
    stored_prompts = all_docs["documents"]
    stored_embeddings = all_docs["embeddings"]

    # Exact match check
    if user_input in stored_prompts:
        print(f"User Input: {user_input}")
        print(f"- Exact match found: {user_input}")
        vec = model.encode(user_input, normalize_embeddings=True)
        print(f"- Vector (first 8 values): {vec[:8]} ...")
        return

    # Semantic similarity
    query_vec = model.encode(user_input, normalize_embeddings=True)
    results = collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )

    top_doc = results['documents'][0][0]
    distance = results['distances'][0][0]
    similarity = 1 - distance

    # Retrieve matched vector from stored embeddings
    doc_index = stored_prompts.index(top_doc)
    matched_vec = stored_embeddings[doc_index]

    print(f"User Input: {user_input}")
    print(f"- Closest Match: {top_doc}")
    print(f"- Similarity Score: {similarity:.2f}")
    print(f"- Query Vector (first 8 values): {query_vec[:8]} ...")
    print(f"- Matched Vector (first 8 values): {matched_vec[:8]} ...")

    if similarity < threshold:
        print("- Note: Low similarity — might not be a perfect match.")


In [13]:
query_prompt(" sushrutha .")
query_prompt("machine learning.")
query_prompt("Prepare 10 AI-related job interview questions.")
query_prompt("Prepare a short story suitable for children aged ")


User Input:  sushrutha .
- Closest Match: Generate SEO-friendly blog titles for a given topic.
- Similarity Score: -0.71
- Query Vector (first 8 values): [ 0.01530063 -0.00396334 -0.00167272 -0.04166892  0.03251288 -0.01215814
 -0.03336141  0.00044055] ...
- Matched Vector (first 8 values): [ 0.06486309  0.00179089 -0.02919099 -0.03538771 -0.00034009  0.00166338
  0.02583491 -0.01011947] ...
- Note: Low similarity — might not be a perfect match.
User Input: machine learning.
- Closest Match: Explain supervised learning vs unsupervised learning.
- Similarity Score: -0.18
- Query Vector (first 8 values): [-0.02040896  0.08269178 -0.08080237  0.01793541 -0.05118631  0.02816961
 -0.01285384  0.03884921] ...
- Matched Vector (first 8 values): [-0.02353059 -0.04476804 -0.03104034  0.0149153  -0.02765055  0.01102278
  0.07930189 -0.02876867] ...
- Note: Low similarity — might not be a perfect match.
User Input: Prepare 10 AI-related job interview questions.
- Closest Match: Create technical i